In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :hNB;
const FLTP = Float64
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
IJuliaCell(vis)

In [ ]:
setobject!(vis[:hbunny],PointCloud(vertices(half_b),fill(convert(RGB{Float32},(parse(Colorant, "darkred"))),length(vertices(half_b)))))
setobject!(vis[:rotbunny],PointCloud(vertices(half_b_rot),fill(convert(RGB{Float32},(parse(Colorant, "olivedrab2"))),length(vertices(half_b_rot)))))

In [ ]:
estiMeshe = makeMeshfromArray([[1,1,1],[2,2,2],[3,3,3]])

In [ ]:
reggeliMeshe = makeMeshfromMatrix(rand(100,3))

In [ ]:
revMeshe = makeMatrixFromMesh(reggeliMeshe)

In [ ]:
function preProcessMeshes(reading,reference)
    
    return reading_iter_mat, reference_iter_mat
end

In [ ]:
function registrateMeshes(reading,reference)
    preProcess()
    iterate()
    postProcess()
    return reg_transformation
end